In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm,colors
import seaborn as sns
from myutils.utils import *

In [ ]:
import folium
from branca.element import Figure
import IPython.display as display
from PIL import Image
WINNIPEG = [49.88366050119829, -97.14581222292078]

In [ ]:
LANE_CLOSURE = import_data("LANE_CLOSURE")
LANE_CLOSURE

In [ ]:
STOPS = import_data("STOPS")
STOPS

In [ ]:
SITES = import_data("SITES")
SITES

In [ ]:
fig=Figure(height=1050,width=1250)
m=folium.Map(location=WINNIPEG,tiles='cartodbpositron',zoom_start=12)
fig.add_child(m)

# Plotting Traffic count sites
f1=folium.FeatureGroup("Traffic Count Sites")
for i in SITES.index:
    site = SITES.loc[i]
    folium.Marker(location=list(site[["Lat","Long"]]),tooltip=f'{site["Street"]}',popup=f"{site['Site']}",icon=folium.Icon(color="red")).add_to(f1)
f1.add_to(m)

# Plotting Bus stops
f2=folium.FeatureGroup("Bus stops")
for j in STOPS.index:
    stop = STOPS.loc[j]
    folium.Circle(radius=4, location=list(stop[["Lat","Long"]]),tooltip=f'{stop["Stop Number"]}',popup=f"{stop['Stop Name']}",color="#3ea6d6",fill=True).add_to(f2)
f2.add_to(m)

# Plotting Road closure
color = {"Yes":"red","No":"orange"}
f3=folium.FeatureGroup("Road closure")
for j in LANE_CLOSURE.index:
    lane = LANE_CLOSURE.iloc[j]
    for i in range(len(lane["Geometry"])):
        line_1=folium.vector_layers.PolyLine(lane["Geometry"][i],popup=f'<b>Direction:</b> {", ".join(lane["Direction"])}',tooltip=f'{lane["Street"]}',color=color[lane["Complete Closure"]],weight=4).add_to(f3)
f3.add_to(m)
folium.LayerControl().add_to(m)
m

In [ ]:
# display.display(Image.open('plots/overview.png'))

In [ ]:
# display.display(Image.open('plots/close_up.png'))

In [ ]:
ROAD_NETWORK = import_data("ROAD_NETWORK")
ROAD_NETWORK["Number of Lanes"] = ROAD_NETWORK["Number of Lanes"].fillna(0).astype(int)
ROAD_NETWORK

In [ ]:
# - One-way values are as follows:
# 0 – The street is a two-way street
# 1 – The one-way direction is opposite the digitized direction
# 2 – The one-way direction is the same as the digitized direction

In [ ]:
# cmap = cm.get_cmap("coolwarm",6)
# color=colors.rgb2hex(cmap(road["Number of Lanes"]))

In [ ]:
fig=Figure(height=1050,width=1250)
# fig=Figure(height=750,width=900)

m=folium.Map(location=WINNIPEG,tiles='cartodbpositron',zoom_start=12)
# folium.TileLayer('cartodbdark_matter').add_to(m)
fig.add_child(m)

color = {50:"grey",0:"lightblue",1:"#ffaf87",2:"#ffaf87"}
f3=folium.FeatureGroup("Road Network")
for j in ROAD_NETWORK.index:
    road = ROAD_NETWORK.loc[j]
    folium.vector_layers.PolyLine(road["Location"],popup=f'<b>Number of Lanes</b> {road["Number of Lanes"]}',tooltip=f'{road["Street"]}',color=color[road["Oneway"]],weight=3).add_to(f3)
f3.add_to(m)
folium.LayerControl().add_to(m)
m

In [ ]:
# display.display(Image.open('plots/road_network.jpg'))

In [ ]:
num_colors = 12
cmap = cm.get_cmap("rainbow",num_colors)
cmap

In [ ]:
fig=Figure(height=750,width=900)
m=folium.Map(location=WINNIPEG,tiles='cartodbpositron',zoom_start=12)
fig.add_child(m)

# Plotting Traffic count sites
f1=folium.FeatureGroup("Traffic Count Sites")
for i in SITES.index:
    site = SITES.loc[i]
    folium.Marker(location=list(site[["Lat","Long"]]),tooltip=f'{site["Street"]}',popup=f"{site['Site']}",icon=folium.Icon(color="red")).add_to(f1)
f1.add_to(m)

# Plotting Bus stops
f2=folium.FeatureGroup("Bus stops")
for j in STOPS.index:
    stop = STOPS.loc[j]
    folium.Circle(radius=4, location=list(stop[["Lat","Long"]]),tooltip=f'{stop["Stop Number"]}',popup=f"{stop['Stop Name']}",color="#3ea6d6",fill=True).add_to(f2)
f2.add_to(m)

# Plotting Road closure
color = {"Yes":"red","No":"orange"}
f3=folium.FeatureGroup("Road closure")
for j in LANE_CLOSURE.index:
    lane = LANE_CLOSURE.iloc[j]
    col=colors.rgb2hex(cmap(int(lane.name)%num_colors))
    # folium.Marker(location=list(lane[["Lat","Long"]]),tooltip=f'{site["Street"]}',popup=f"{site['Site']}",icon=folium.Icon(color=col)).add_to(f3)
    folium.Circle(radius=10,location=list(lane[["Lat","Long"]]),tooltip=f'{int(lane.name)}',popup=f"{col}",color=col, fill =True).add_to(f3)
    for i in range(len(lane["Geometry"])):
        for k in range(len(lane["Geometry"][i])):
            folium.Circle(radius=5,location=list(lane["Geometry"][i][k]),tooltip=f'{int(lane.name)}',popup=f"{col}",color=col).add_to(f3)
        line_1=folium.vector_layers.PolyLine(lane["Geometry"][i],popup=f'<b>Direction:</b> {", ".join(lane["Direction"])}',tooltip=f'{lane["Street"]}',color=col,weight=4).add_to(f3)
f3.add_to(m)
folium.LayerControl().add_to(m)
# m.add_child(folium.LatLngPopup())
# m.add_child(folium.ClickForMarker(popup="Waypoint"))

m

1. Algorithms
    - Find the polygon that contains the lane closure segment(s)
    - Determine whether a point (bus stop) belongs in a polygon 
    - Polar coordinates 
    - Determine directly and indirectly affected bus stops

In [ ]:
# Python 3 program to calculate Distance Between Two Points on Earth, courtesy of https://www.geeksforgeeks.org/program-distance-two-points-earth/
from math import radians, cos, sin, asin, sqrt
def distance(pt1,pt2):
    # Convert from degrees to radians.
    lat1 = radians(pt1[0])
    lon1 = radians(pt1[1])
    lat2 = radians(pt2[0])
    lon2 = radians(pt2[1])
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6378100
    # calculate the result
    return(c * r)

In [ ]:
s1 = tuple(SITES.iloc[0][["Lat","Long"]])
s2 = tuple(SITES.iloc[1][["Lat","Long"]])


In [ ]:
distance(s1,s2)

To do: 
1. Convert from meters & angles to coordinates
2. How to know which bus stops affected by which route
    - Using Polygon. Point in Polygon
3. 

